# Consultas SQL Simples com DuckDB

**NÃO precisa de Docker, Spark ou Delta Lake!**

DuckDB lê Parquet direto e permite consultas SQL.

In [ ]:
# Instalar DuckDB (só precisa fazer uma vez)
!pip install -q duckdb pandas minio pyarrow

In [ ]:
import duckdb
import pandas as pd
from minio import Minio
import io

# Configurações MinIO
MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ROOT_USER = "admin"
MINIO_ROOT_PASSWORD = "1q2w3e4r"
BUCKET_NAME = "govbr"

# Cliente MinIO
minio_client = Minio(
    MINIO_SERVER_URL,
    access_key=MINIO_ROOT_USER,
    secret_key=MINIO_ROOT_PASSWORD,
    secure=True
)

def read_parquet_from_minio(path):
    """Lê arquivo Parquet do MinIO"""
    try:
        response = minio_client.get_object(BUCKET_NAME, path)
        df = pd.read_parquet(io.BytesIO(response.read()))
        response.close()
        response.release_conn()
        return df
    except Exception as e:
        print(f"❌ Erro ao ler {path}: {e}")
        return None

# Criar conexão DuckDB
conn = duckdb.connect()

print("✅ DuckDB conectado!")

## Carregar Dados

In [ ]:
# Carregar dados da camada Ouro
df_estados = read_parquet_from_minio("ouro/dim_estados_enriquecida/dt=20251114/data.parquet")
df_municipios = read_parquet_from_minio("ouro/dim_municipios_enriquecida/dt=20251114/data.parquet")
df_bpc = read_parquet_from_minio("ouro/fato_bpc_enriquecido/dt=20251114/data.parquet")

# Registrar como tabelas no DuckDB
if df_estados is not None:
    conn.register('estados', df_estados)
    print(f"✅ Estados: {len(df_estados)} registros")

if df_municipios is not None:
    conn.register('municipios', df_municipios)
    print(f"✅ Municípios: {len(df_municipios)} registros")

if df_bpc is not None:
    conn.register('bpc', df_bpc)
    print(f"✅ BPC: {len(df_bpc)} registros")

## Consultas SQL

In [ ]:
# Consulta 1: Listar estados
conn.execute("SELECT * FROM estados LIMIT 10").fetchdf()

In [ ]:
# Consulta 2: Join entre tabelas
conn.execute("""
    SELECT 
        e.uf_sigla,
        e.uf_nome,
        SUM(b.valor) as total_bpc,
        SUM(b.quantidade_beneficiados) as total_beneficiados
    FROM bpc b
    JOIN estados e ON b.uf_sigla = e.uf_sigla
    GROUP BY e.uf_sigla, e.uf_nome
    ORDER BY total_bpc DESC
""").fetchdf()

In [ ]:
# Consulta 3: Agregação por região
conn.execute("""
    SELECT 
        regiao_nome,
        COUNT(*) as total_municipios
    FROM municipios
    WHERE regiao_nome IS NOT NULL
    GROUP BY regiao_nome
    ORDER BY total_municipios DESC
""").fetchdf()

In [ ]:
# Consulta 4: Top municípios por BPC
conn.execute("""
    SELECT 
        nome_municipio,
        uf_sigla,
        valor,
        quantidade_beneficiados
    FROM bpc
    ORDER BY valor DESC
    LIMIT 10
""").fetchdf()

## Usando Sintaxe Mais Simples

In [ ]:
# Método mais simples usando .sql()
conn.sql("SELECT * FROM estados WHERE populacao > 10000000").show()

In [ ]:
# Converter resultado para DataFrame do Pandas
resultado = conn.sql("SELECT uf_sigla, COUNT(*) as total FROM municipios GROUP BY uf_sigla").df()
print(resultado)